# Hello Clustering

**K-mean clustering**


Concretely, if we are given N data points, x1, x2, ..., xN , and we would like
to form K clusters. We do the following;
1. **Initialization**: Pick K random data points as K centroid locations c1,
c2, ..., cK.
2. **Assign**: For each data point k, find the closest centroid. Assign that
data point to the centroid. The distance used is typically Euclidean distance.
3. **Update**: For each centroid, calculate the mean from the data points
assigned to it.
4. **Repeat**: repeat step 2 and 3 until the centroids stop changing (convergence).

In [0]:
import matplotlib.pyplot as plt

points = [(1, 2), (3, 3), (2, 2), (8, 8), (6, 6),
            (7, 7), (-3, -3), (-2, -4), (-7, -7)]

def plot(clusters, title):
  DOT_COLORS = 'rgbcmyk'
  fig = plt.figure()
  plt.title(title)
  for i in range(len(clusters)):
    for point in clusters[i]:
      plt.plot(point[0], point[1], 'ro', c=DOT_COLORS[i])
  fig.show()

def dist(p1, p2):
    return ((p1[0] - p2[0]) ** 2 + (p1[1] - p2[1]) ** 2) ** 0.5

def print_point(p):
    print('(', p[0], ',', p[1], ')', end=' ')
  
def k_mean(start_centroids, ITER_COUNT = 4, debug = False):
  centroids = start_centroids
  k = len(centroids)

  def print_assign(section):
      print('Assign Step')
      for i in range(k):
          print('Cluster', i, ':')
          for point in section[i]:
              print_point(point)
          print()
      print()

  def update_centroids(section):
      new_centroid = []
      for i in range(k):
          x = [pt[0] for pt in section[i]]
          y = [pt[1] for pt in section[i]]
          new_centroid.append((sum(x)/len(x), sum(y)/len(y)))
      return new_centroid


  def print_centroids(centroids):
      print("Centroids: ", end='' )
      for c in centroids:
        print_point(c)
      print()

  for i in range(ITER_COUNT):
      if debug: 
        print("Iter", i)
        print_centroids(centroids)
      section = []
      for j in range(k):
          section.append([])
      for point in points:
          section[min([(dist(point, centroids[i]), i) for i in range(k)])[1]].append(point)
      if debug:
        print_assign(section)
      centroids = update_centroids(section)
  
  return { "centroids": centroids, "clusters": section }

# T4
T4 = k_mean([(3, 3), (2, 2), (-3, -3)])
# T5
T5 = k_mean([(-3, -3), (2, 2), (-7, -7)])

plot(T4['clusters'], 'T4')
plot(T5['clusters'], 'T5')

We can use explained variance to measure clustering quality.

explained variance = between-cluster var / all-data var


In [0]:
def all_point_centroid():
  x = [pt[0] for pt in points]
  y = [pt[1] for pt in points]
  return (sum(x)/len(x), sum(y)/len(y))

all_centroid = all_point_centroid()

def all_data_variance():
  s = 0
  for pt in points:
    s += dist(pt, all_centroid) ** 2
  return s / (len(points) - 1)
    
def between_cluster_variance(centroids, clusters):
  s = 0
  for i in range(len(centroids)):
    s += len(clusters[i]) * dist(centroids[i], all_centroid) ** 2
  return s / (len(points) - 1)

def explained_var(k_mean_result): # { 'centroids': [(x, y)], 'clusters': [[(x, y)]] }
  return between_cluster_variance(k_mean_result['centroids'], k_mean_result['clusters']) / all_data_variance()
  
# T6
T4_res = k_mean([(3, 3), (2, 2), (-3, -3)])
T5_res = k_mean([(-3, -3), (2, 2), (-7, -7)])

print(explained_var(T4_res)) # 0.9298618490967058
print(explained_var(T5_res)) # 0.8138947927736452

The result tells us that T4 is better than T5

To determine the best K for this question, we can use Elbow method though it isn't so accurate.
Elbow method chooses K where increasing complexity doesn't yield much in return. (ie. minimal K that explains at least 95% of the all-data variance)

In [0]:
# OT1
import matplotlib.pyplot as plt
from random import sample
import numpy as np

MAX_K = 10
ITER_PER_K = 50

# Plotting
exp_vars = []
num_of_cluster = [k for k in range(1, MAX_K)]
fig = plt.figure()

for k in num_of_cluster:
  s = 0
  tries = 0
  for i in range(ITER_PER_K):
    try:
      start_pts = sample(points, k)
      s += explained_var(k_mean(start_pts, 100))
      tries += 1
    except:
      pass
  exp_vars.append(s / tries)

print(exp_vars)

plt.plot(num_of_cluster, exp_vars, marker='o')
line_95p = np.linspace(0, MAX_K, 1000)
plt.plot(line_95p, np.zeros(1000) + 0.95) # 95% line
plt.xlabel('Number of clusters (K)')
plt.ylabel('Explained Variance')
plt.show()
  

From the result above, K=4 is the best K for this set of points.

# My Heart Will Go On

**Load data set and test set** and fill all NaN value with median (or mode) before training the model

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math

train_url = "http://s3.amazonaws.com/assets.datacamp.com/course/Kaggle/train.csv"
train = pd.read_csv(train_url) #training set
test_url = "http://s3.amazonaws.com/assets.datacamp.com/course/Kaggle/test.csv"
test = pd.read_csv(test_url) #test set

# T7: Find Median
print('Median =', train['Age'].median()) # 28.0
train['Age'] = train['Age'].fillna(train['Age'].median())

# Assign embarked value to number
MAPPING_CONST = { 'Embarked': { 'S': 0, 'C': 1, 'Q': 2 }, 'Sex': { 'male': 0, 'female': 1 } }
for key in MAPPING_CONST:
  for val in MAPPING_CONST[key]:
    train.loc[train[key] == val, key] = MAPPING_CONST[key][val]
    test.loc[test[key] == val, key] = MAPPING_CONST[key][val]

# T8: Find Mode
print('Embarked Mode =', train.mode()['Embarked'][0]) # 0 (S)
train['Embarked'] = train['Embarked'].fillna(train.mode()['Embarked'][0])
print('Sex Mode =', train.mode()['Sex'][0]) # 0 (male)
train['Sex'] = train['Sex'].fillna(train.mode()['Sex'][0])

# Fill NaN for test data
test['Embarked'] = test['Embarked'].fillna(test.mode()['Embarked'][0])
test['Sex'] = test['Sex'].fillna(test.mode()['Sex'][0])
test['Age'] = test['Age'].fillna(test['Age'].median())

## Logistic Regression

In [0]:
# T9 Logistic Regression
features = ["Pclass","Sex","Age","Embarked"]

# Initiate plot of loss function
fig = plt.figure()
loss = []
iter_count = []
PLOT_FILE_NAME = 'loss.png'
plt.xlabel('Number of iterations')
plt.ylabel('Loss')

def sigmoid(theta, x_i):
  dot = np.dot(theta, x_i)
  return 1 / (1 + math.exp(-dot))

def updated_theta(theta, data, result, rate):
  new_theta = np.array(theta)
  for j in range(len(features) + 1):
    s = 0
    for i in range(data.shape[0]):
      s += (result[i] - sigmoid(np.array([theta]), np.array([data[i]]).T)) * data[i][j]
    new_theta[j] = theta[j] + rate * s
  return new_theta

def logistic_regression(rate = 0.00001, ITER_COUNT = -1):
  raw_data = np.array(train[features].values, dtype=int)
  data = np.append(np.ones((raw_data.shape[0], 1)), raw_data, axis = 1)
  result = np.array(train['Survived'].values, dtype=int)
  theta = np.zeros(len(features) + 1)
  i = 0
  while i < ITER_COUNT or ITER_COUNT == -1:
    theta = updated_theta(theta, data, result, rate)
    
    # Logging every 100 iter
    if i % 100 == 0: 
      print('Iter', i, ':', theta)
      loss.append(loss_function(theta, data, result))
      iter_count.append(i)
      plt.plot(iter_count, loss)
      fig.savefig(PLOT_FILE_NAME)
      
    i += 1
  return theta

def loss_function(theta, data, result):
  s = 0
  for i in range(data.shape[0]):
    hyp = sigmoid(np.array([theta]), np.array([data[i]]).T)
    s -= result[i] * math.log(hyp) + (1 - result[i]) * math.log(1 - hyp)
  return s / data.shape[0]
  
model = logistic_regression()

Write the solution in .csv

In [0]:
OUTPUT_FILE_NAME = 'solution.csv'

def test_model(theta, THRESHOLD = 0.5):
  # Result DataFrame
  result = pd.DataFrame(columns=['PassengerId', 'Survived'])
  
  raw_data = np.array(test[features].values, dtype=int)
  data = np.append(np.ones((raw_data.shape[0], 1)), raw_data, axis = 1)
  for i in range(data.shape[0]):
    train_sol = sigmoid(np.array([theta]), np.array([data[i]]).T)
    result.loc[i] = [test['PassengerId'][i]] + [int(train_sol >= THRESHOLD)]
  
  return result

model = np.array([2.07070249,-1.19638948,2.57579964,-0.03372561,0.32077026])
result = test_model(model)
result.to_csv(OUTPUT_FILE_NAME, index=False)

Try adding some higher order features.

In [0]:
# OT4
high_order_features = ["Pclass"]
features = ["Pclass","Sex","Age","Embarked"]

for feature in high_order_features:
  new_fname = feature + "2"
  train[new_fname] = train[feature] ** 2
  test[new_fname] = test[feature] ** 2
  features.append(new_fname)

OUTPUT_FILE_NAME = 'solution_higher_order.csv'

print(features)
model = logistic_regression(0.000001)
# [ 0.66371864  0.3197872   2.55857657 -0.03117537  0.37376845 -0.36377816]
result = test_model(model)
result.to_csv(OUTPUT_FILE_NAME, index=False)

Try using only Sex and Age features.

In [0]:
# OT5
features = ["Sex", "Age"]
OUTPUT_FILE_NAME = 'solution_sex_age.csv'

# print(features)
# model = logistic_regression(0.000001)
model = np.array([-1.29902669,2.49762116,-0.00513416])
result = test_model(model)
result.to_csv(OUTPUT_FILE_NAME, index=False)

## Linear Regression
Redo by using linear regression (Gradient descent) instread.

In [0]:
# OT2
def updated_theta_linear(theta, data, result, rate):
  new_theta = np.array(theta)
  for j in range(len(features) + 1):
    s = 0
    for i in range(data.shape[0]):
      s += (result[i] - np.dot(np.array([theta]), np.array([data[i]]).T)) * data[i][j]
    new_theta[j] = theta[j] + rate * s
  return new_theta

def linear_regression(rate = 0.0000001, ITER_COUNT = -1):
  raw_data = np.array(train[features].values, dtype=int)
  data = np.append(np.ones((raw_data.shape[0], 1)), raw_data, axis = 1)
  result = np.array(train['Survived'].values, dtype=int)
  theta = np.zeros(len(features) + 1)
  i = 0
  while i < ITER_COUNT or ITER_COUNT == -1:
    theta = updated_theta_linear(theta, data, result, rate)
    
    # Logging every 100 iter
    if i % 100 == 0: 
      print('Iter', i, ':', theta)
      
    i += 1
  return theta

model = linear_regression()
# Iter 375800 : [ 0.60398843 -0.14915006  0.52049079 -0.00288894  0.05109822]

Write linear regression solution in .csv

In [0]:
OUTPUT_FILE_NAME = 'linear_solution.csv'
def test_model_linear(theta, THRESHOLD = 0.5):
  # Result DataFrame
  result = pd.DataFrame(columns=['PassengerId', 'Survived'])
  
  raw_data = np.array(test[features].values, dtype=int)
  data = np.append(np.ones((raw_data.shape[0], 1)), raw_data, axis = 1)
  for i in range(data.shape[0]):
    train_sol = np.dot(np.array([theta]), np.array([data[i]]).T)
    result.loc[i] = [test['PassengerId'][i]] + [int(train_sol >= THRESHOLD)]
  
  return result

model = np.array([0.60398843,-0.14915006,0.52049079,-0.00288894,0.05109822])
result = test_model_linear(model)
result.to_csv(OUTPUT_FILE_NAME, index=False)

Redo by using linear regression (matrix inversion)

theta = (X^TX)^-1X^TY

In [0]:
# OT3
from numpy.linalg import inv

def solve_linear():
  raw_data = np.array(train[features].values, dtype=int)
  data = np.append(np.ones((raw_data.shape[0], 1)), raw_data, axis = 1)
  result = np.array(train['Survived'].values, dtype=int)
  return inv(np.dot(data.T, data)).dot(data.T).dot(result)

print(solve_linear())
# [ 0.776742   -0.18848969  0.4908994  -0.00505977  0.04907325]

We found that the weights from two method are similar.

We'll report MSE of the difference between two weights.

In [0]:
weight_gradient =  [0.60398843,-0.14915006,0.52049079,-0.00288894,0.05109822]
weight_inverse = [0.776742,-0.18848969,0.4908994,-0.00505977,0.04907325]

def MSE(a, b):
  return sum([(a[i] - b[i]) ** 2 for i in range(len(a))]) / len(a)

print(MSE(weight_gradient,weight_inverse))
# 0.006455173160960742